In [162]:
url = 'https://www-genesis.destatis.de/genesis/online?sequenz=tabelleErgebnis&selectionname=23211-0002#abreadcrumb'

In [163]:
from bs4 import BeautifulSoup 
import pandas as pd
import requests
import re

In [164]:
req = requests.get(url)
req

<Response [200]>

In [165]:
html = BeautifulSoup(req.content, 'html.parser')
html

<!DOCTYPE html>
<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="index,follow,noarchive,noodp" name="robots"/>
<meta content="30 Days" name="revisit-after"/>
<meta content="GENESIS-Online" name="generator"/>
<meta content="Text" name="DC.type"/>
<meta content="text/html" name="DC.format"/>
<meta content="general public" name="DC.audience"/>
<meta content="de" name="language"/>
<meta content="de" name="DC.language"/>
<meta content="2023-08-01T12:49:10+1:00" name="date"/>
<meta content="2023-08-01T12:49:10+1:00" name="DC.date"/>
<meta content="https://www-genesis.destatis.de:443/genesis//online/1690886950058" name="DC.identifier"/>
<meta content="https://www-genesis.destatis.de:443/genesis//online?Menu=Impressum" name="DC.rights"/>
<meta content="GENESIS-Online ist eine Datenbank, die tief gegliederte Ergebnisse der amtlichen Statistik enthält. Sie wird kontinuierlich ausgebaut. Der Tabellenabruf erfolgt unentgeltlich und kann va

In [166]:
table = html.find('table', attrs={'id':'ET'})
table

<table class="ergebnistabelle" id="ET" summary="Die Tabelle 23211-0002 enthält aktuelle Daten zur Statistik - Todesursachenstatistik - mit Stand vom 01.08.2023"><caption><span style="white-space:nowrap">Gestorbene: Deutschland, Jahre, Todesursachen, Geschlecht</span></caption><thead><tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etueberschrift" colspan="4" id="Z2S1" style="text-align: left"><span style="white-space:nowrap"><input name="option$$$01923211" title="Anzeigen von Informationen zu Statistik mit dem Namen '23211'" type="submit" value="Todesursachenstatistik"><br>Deutschland<br/><input name="option$$$003BEV002" title="Anzeigen von Informationen zu Merkmal mit dem Namen 'BEV002'" type="submit" value="Gestorbene"> (Anzahl)</input></br></input></span></th></tr><tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etkopfvorspalte" colspan="1" id="Z3S1" rowspan="2"><span style="white-space:nowrap">Todesursachen</span></th><th class="schemaBorderColorLevel3 schemaBgC

In [167]:
#list_id_th = ['Z2S1','Z3S1','Z3S2','Z4S2','Z4S3','Z4S4']
if table:
    # Cari semua elemen <th> dalam tabel
    semua_judul_kolom = table.find_all('th')

    # List ID elemen <th> yang diinginkan (123, 456, 789, 111, 222)
    list_id_th = ['Z2S1','Z3S1','Z4S2','Z4S3','Z4S4']

    # Filter elemen <th> yang memiliki ID sesuai dengan list_id_th
    hasil_scrapping = [judul_kolom for judul_kolom in semua_judul_kolom if judul_kolom.get('id') in list_id_th]

    # Lakukan sesuatu dengan setiap elemen <th> yang ditemukan
    for judul_kolom in hasil_scrapping:
        print("ID:", judul_kolom['id'])
        print("Judul kolom:", judul_kolom.text)
else:
    print("Tabel dengan ID 'ET' tidak ditemukan.")

ID: Z2S1
Judul kolom: Deutschland (Anzahl)
ID: Z3S1
Judul kolom: Todesursachen
ID: Z4S2
Judul kolom: männlich
ID: Z4S3
Judul kolom: weiblich
ID: Z4S4
Judul kolom: Insgesamt


In [168]:
columns_n = [col.text for col in hasil_scrapping]
columns_n

['Deutschland\xa0(Anzahl)',
 'Todesursachen',
 'männlich',
 'weiblich',
 'Insgesamt']

In [169]:
columns = [re.sub('\xa0','',col) for col in columns_n]
columns

['Deutschland(Anzahl)', 'Todesursachen', 'männlich', 'weiblich', 'Insgesamt']

In [170]:
title = columns.pop(0)
print(title, columns)

Deutschland(Anzahl) ['Todesursachen', 'männlich', 'weiblich', 'Insgesamt']


In [171]:
table.find_all('tbody')

[<tbody><tr><th class="schemaBorderColorLevel3 etuntertitel" colspan="4" id="Z5S1">2021</th></tr><tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etvorspalte" id="Z6S1">Bestimmte infektiöse und parasitäre Krankheiten</th><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S2" style="white-space: nowrap">7 355</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S3" style="white-space: nowrap">7 515</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S4" style="white-space: nowrap">14 870</td></tr><tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etvorspalte" id="Z7S1">Tuberkulose einschließlich ihrer Folgezustände</th><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S2" style="white-space: nowrap">161</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S3" style="white-space: nowrap">106</td><td class="schemaBorderColorLevel3 etzelle" heade

In [172]:
#ada berapa tbody
len(table.find_all('tbody'))

1

In [173]:
rows = table.find_all('tbody')[0].find_all('tr') #untuk dapetin row nya saja
rows

[<tr><th class="schemaBorderColorLevel3 etuntertitel" colspan="4" id="Z5S1">2021</th></tr>,
 <tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etvorspalte" id="Z6S1">Bestimmte infektiöse und parasitäre Krankheiten</th><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S2" style="white-space: nowrap">7 355</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S3" style="white-space: nowrap">7 515</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S4" style="white-space: nowrap">14 870</td></tr>,
 <tr><th class="schemaBorderColorLevel3 schemaBgColorLevel1 etvorspalte" id="Z7S1">Tuberkulose einschließlich ihrer Folgezustände</th><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S2" style="white-space: nowrap">161</td><td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S3" style="white-space: nowrap">106</td><td class="schemaBorderColorLevel3 etzelle" header

In [174]:
table.find_all('tbody')[0].find_all('tr')[1].text

'Bestimmte infektiöse und parasitäre Krankheiten7 3557 51514 870'

In [175]:
for row in table.find_all('tbody')[0].find_all('tr'):
    print(row.find_all('td'))

[]
[<td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S2" style="white-space: nowrap">7 355</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S3" style="white-space: nowrap">7 515</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z6S1 Z3S2 Z4S4" style="white-space: nowrap">14 870</td>]
[<td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S2" style="white-space: nowrap">161</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S3" style="white-space: nowrap">106</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z7S1 Z3S2 Z4S4" style="white-space: nowrap">267</td>]
[<td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z8S1 Z3S2 Z4S2" style="white-space: nowrap">8</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2S1 Z5S1 Z8S1 Z3S2 Z4S3" style="white-space: nowrap">7</td>, <td class="schemaBorderColorLevel3 etzelle" headers="Z2

In [176]:
data_dict = []

# Loop melalui masing-masing elemen <tr>
for row in rows:
    # Cari elemen <th> dalam elemen <tr>
    th_element = row.find('th', class_='schemaBgColorLevel1')
    if th_element:
        # Ambil teks dari elemen <th> sebagai judul kolom
        judul_kolom = th_element.text.strip()
        
        # Cari elemen <td> dalam elemen <tr>
        td_elements = row.find_all('td', class_='etzelle')
        
        # Ambil teks dari setiap elemen <td> sebagai nilai-nilai terkait
        nilai_values = [td.text.strip() for td in td_elements]
        
        # Simpan data dalam dictionary
        data = [judul_kolom] + nilai_values
        data_dict.append(data)

# Cetak data yang diambil
for data in data_dict:
    print(data)

['Bestimmte infektiöse und parasitäre Krankheiten', '7 355', '7 515', '14 870']
['Tuberkulose einschließlich ihrer Folgezustände', '161', '106', '267']
['Meningokokkeninfektion', '8', '7', '15']
['Virushepatitis', '307', '204', '511']
['HIV-Krankheit', '176', '42', '218']
['Neubildungen', '127 968', '109 192', '237 160']
['Bösartige Neubildungen (BN)', '124 006', '105 062', '229 068']
['', '3 796', '1 421', '5 217']
['', '4 444', '1 368', '5 812']
['', '4 976', '3 320', '8 296']
['', '7 820', '7 252', '15 072']
['', '4 744', '3 049', '7 793']
['', '16 708', '14 286', '30 994']
['', '5 455', '2 692', '8 147']
['', '9 570', '9 602', '19 172']
['', '28 332', '17 608', '45 940']
['', '27 200', '17 401', '44 601']
['', '2 406', '1 700', '4 106']
['Bösartiges Melanom der Haut', '1 692', '1 236', '2 928']
['', '157', '18 479', '18 636']
['', '25 989', '16 181', '42 170']
['', '-', '1 535', '1 535']
['', '-', '2 634', '2 634']
['', '-', '5 379', '5 379']
['', '15 379', '-', '15 379']
['', '3 0

In [177]:
data_dict #sudah tidak nested list

[['Bestimmte infektiöse und parasitäre Krankheiten',
  '7 355',
  '7 515',
  '14 870'],
 ['Tuberkulose einschließlich ihrer Folgezustände', '161', '106', '267'],
 ['Meningokokkeninfektion', '8', '7', '15'],
 ['Virushepatitis', '307', '204', '511'],
 ['HIV-Krankheit', '176', '42', '218'],
 ['Neubildungen', '127 968', '109 192', '237 160'],
 ['Bösartige Neubildungen (BN)', '124 006', '105 062', '229 068'],
 ['', '3 796', '1 421', '5 217'],
 ['', '4 444', '1 368', '5 812'],
 ['', '4 976', '3 320', '8 296'],
 ['', '7 820', '7 252', '15 072'],
 ['', '4 744', '3 049', '7 793'],
 ['', '16 708', '14 286', '30 994'],
 ['', '5 455', '2 692', '8 147'],
 ['', '9 570', '9 602', '19 172'],
 ['', '28 332', '17 608', '45 940'],
 ['', '27 200', '17 401', '44 601'],
 ['', '2 406', '1 700', '4 106'],
 ['Bösartiges Melanom der Haut', '1 692', '1 236', '2 928'],
 ['', '157', '18 479', '18 636'],
 ['', '25 989', '16 181', '42 170'],
 ['', '-', '1 535', '1 535'],
 ['', '-', '2 634', '2 634'],
 ['', '-', '5 3

In [178]:
df = pd.DataFrame(data_dict, columns=columns)
df.sample(10)

,Todesursachen,männlich,weiblich,Insgesamt
52,"COVID-19, Virus nachgewiesen",38 862,31 897,70 759
16,,27 200,17 401,44 601
43,Sonstige Krankheiten des Endokards,6 560,8 428,14 988
18,Bösartiges Melanom der Haut,1 692,1 236,2 928
69,Plötzlicher Kindstod,47,35,82
8,,4 444,1 368,5 812
19,,157,18 479,18 636
51,Asthma,406,643,1 049
75,Ertrinken und Untergehen,281,111,392
30,,2 078,2 632,4 710


In [179]:
df.replace({' ':'','-':'0'}, regex = True, inplace=True)

In [180]:
df.head(20)

,Todesursachen,männlich,weiblich,Insgesamt
0,BestimmteinfektiöseundparasitäreKrankheiten,7355,7515,14870
1,TuberkuloseeinschließlichihrerFolgezustände,161,106,267
2,Meningokokkeninfektion,8,7,15
3,Virushepatitis,307,204,511
4,HIV0Krankheit,176,42,218
5,Neubildungen,127968,109192,237160
6,BösartigeNeubildungen(BN),124006,105062,229068
7,,3796,1421,5217
8,,4444,1368,5812
9,,4976,3320,8296


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Todesursachen  82 non-null     object
 1   männlich       82 non-null     object
 2   weiblich       82 non-null     object
 3   Insgesamt      82 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [191]:
#df.loc[7, 'Todesursachen'] = 'BN der Lippe, der Mundhöhle und des Pharynx'
#df.loc[8, 'Todesursachen'] = 'BN des Ösophagus'
#df.loc[9, 'Todesursachen'] = 'BN des Magens'
#df.loc[10, 'Todesursachen'] = 'BN des Dickdarms'
#df.loc[11, 'Todesursachen'] = 'BN des Rektums, des Anus und am Rektosigmoid'
#df.loc[12, 'Todesursachen'] = 'BN der Leber, der Gallenwege und des Pankreas'
#df.loc[13, 'Todesursachen'] = 'BN der Leber und der intrahepatischen Gallengänge'
#df.loc[14, 'Todesursachen'] = ''
data_tambahan = [
    {'Index': 7, 'Todesursachen': 'BN der Lippe, der Mundhöhle und des Pharynx'},
    {'Index': 8,'Todesursachen': 'BN des Ösophagus'},
    {'Index': 9,'Todesursachen': 'BN des Magens'},
    {'Index': 10,'Todesursachen': 'BN des Dickdarms'},
    {'Index': 11,'Todesursachen': 'BN des Rektums, des Anus und am Rektosigmoid'},
    {'Index': 12,'Todesursachen': 'BN der Leber, der Gallenwege und des Pankreas'},
    {'Index': 13,'Todesursachen': 'BN der Leber und der intrahepatischen Gallengänge'},
    {'Index': 14,'Todesursachen': 'BN des Pankreas'},
    {'Index': 15,'Todesursachen': 'BN d. Larynx. d. Trachea, d. Bronchien u.d. Lunge'},
    {'Index': 16,'Todesursachen': 'BN der Bronchien und der Lunge'},
    {'Index': 17,'Todesursachen': 'Melanom und sonstige bösartige Neubild. der Haut'},
    {'Index': 19,'Todesursachen': 'BN der Brustdrüse'},
    {'Index': 20,'Todesursachen': 'BN der Genital- und Harnorgane'},
    {'Index': 21,'Todesursachen': 'BN der Cervix uteri'},
    {'Index': 22,'Todesursachen': 'BN des Corpus uteri u. Uterus, Teil n.n. bezeichn.'},
    {'Index': 23,'Todesursachen': 'BN des Ovars'},
    {'Index': 24,'Todesursachen': 'BN der Prostata'},
    {'Index': 25,'Todesursachen': 'BN der Niere, ausgenommen Nierenbecken'},
    {'Index': 26,'Todesursachen': 'BN der Harnblase'},
    {'Index': 27,'Todesursachen': 'BN d. lymphat., blutbild. u. verwanten Gewebes'},
    {'Index': 30,'Todesursachen': 'Krankheiten des Blutes u. der blutbildenden Organe'},
    {'Index': 31,'Todesursachen': 'Endokrine, Ernährungs- u. Stoffwechselkranheiten'},
    {'Index': 35,'Todesursachen': 'Psych. u. Verhaltensstörungen d.a.psychotr. Subst.'},
    {'Index': 36,'Todesursachen': 'Krankheiten d. Nervensystems u.d. Sinnesorgane'},
    {'Index': 45,'Todesursachen': 'Schlaganfall, nicht als Blutung oder Infarkt bez.'},
    {'Index': 55,'Todesursachen': 'Ulcus ventriculi, duodeni, pepticum, pept. jejuni'},
    {'Index': 57,'Todesursachen': 'Alkoh.Leberkh.,Chron.Hepatitis,Leberfibr.u.-zirr.'},
    {'Index': 59,'Todesursachen': 'Krankh.des Muskel-Skelett-Systems u. Bindegewebes'},
    {'Index': 64,'Todesursachen': 'Best.Zustände mit Ursprung in der Perinatalperiode'},
    {'Index': 65,'Todesursachen': 'Angeb. Fehlbildungen,Deformitäten,Chromosomenanom'},
    {'Index': 68,'Todesursachen': 'Symptome und abnorme klinische und Laborbefunde'},
    {'Index': 70,'Todesursachen': 'Sonst. ungenau bezeichnete u. unbek. Todesursachen'},
    {'Index': 77,'Todesursachen': 'Akzid. Vergiftung: schädl. Substanzen (inkl. Exp.)'}
]

In [192]:
for tambahan in data_tambahan:
    row_idx = tambahan['Index']  # Indeks baris yang akan diisi
    df.loc[row_idx, 'Todesursachen'] = tambahan['Todesursachen']

In [195]:
df['Todesursachen'].value_counts()

Todesursachen
BestimmteinfektiöseundparasitäreKrankheiten          1
KrankheitendesUrogenitalsystems                      1
Krankh.des Muskel-Skelett-Systems u. Bindegewebes    1
KrankheitenderHautundderUnterhaut                    1
Alkoh.Leberkh.,Chron.Hepatitis,Leberfibr.u.-zirr.    1
                                                    ..
BN der Harnblase                                     1
BN der Niere, ausgenommen Nierenbecken               1
BN der Prostata                                      1
BN des Ovars                                         1
Insgesamt                                            1
Name: count, Length: 82, dtype: int64

In [196]:
df.head(20)

,Todesursachen,männlich,weiblich,Insgesamt
0,BestimmteinfektiöseundparasitäreKrankheiten,7355,7515,14870
1,TuberkuloseeinschließlichihrerFolgezustände,161,106,267
2,Meningokokkeninfektion,8,7,15
3,Virushepatitis,307,204,511
4,HIV0Krankheit,176,42,218
5,Neubildungen,127968,109192,237160
6,BösartigeNeubildungen(BN),124006,105062,229068
7,"BN der Lippe, der Mundhöhle und des Pharynx",3796,1421,5217
8,BN des Ösophagus,4444,1368,5812
9,BN des Magens,4976,3320,8296


In [199]:
df.describe()

,Todesursachen,männlich,weiblich,Insgesamt
count,82,82,82,82
unique,82,79,82,82
top,BestimmteinfektiöseundparasitäreKrankheiten,0,7515,14870
freq,1,4,1,1


In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Todesursachen  82 non-null     object
 1   männlich       82 non-null     object
 2   weiblich       82 non-null     object
 3   Insgesamt      82 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [208]:
df['männlich'] = df['männlich'].astype(int)
df['weiblich'] = df['weiblich'].astype(int)
df['Insgesamt'] = df['Insgesamt'].astype(int)

In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Todesursachen  82 non-null     object
 1   männlich       82 non-null     int64 
 2   weiblich       82 non-null     int64 
 3   Insgesamt      82 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.7+ KB


In [212]:
df.loc[df['männlich']>100000] #Krankheiten des Kreislaufsystems is

,Todesursachen,männlich,weiblich,Insgesamt
5,Neubildungen,127968,109192,237160
6,BösartigeNeubildungen(BN),124006,105062,229068
38,KrankheitendesKreislaufsystems,159846,180773,340619
81,Insgesamt,515559,508128,1023687


In [213]:
df.loc[df['weiblich']>100000]

,Todesursachen,männlich,weiblich,Insgesamt
5,Neubildungen,127968,109192,237160
6,BösartigeNeubildungen(BN),124006,105062,229068
38,KrankheitendesKreislaufsystems,159846,180773,340619
81,Insgesamt,515559,508128,1023687


In [215]:
df.loc[df['weiblich'] > df['männlich']]

,Todesursachen,männlich,weiblich,Insgesamt
0,BestimmteinfektiöseundparasitäreKrankheiten,7355,7515,14870
14,BN des Pankreas,9570,9602,19172
19,BN der Brustdrüse,157,18479,18636
21,BN der Cervix uteri,0,1535,1535
22,"BN des Corpus uteri u. Uterus, Teil n.n. bezei...",0,2634,2634
23,BN des Ovars,0,5379,5379
29,GutartigeNeubildungen,371,396,767
30,Krankheiten des Blutes u. der blutbildenden Or...,2078,2632,4710
31,"Endokrine, Ernährungs- u. Stoffwechselkranheiten",17402,20464,37866
32,Diabetesmellitus,12428,13556,25984


In [221]:
max = df.loc[(df['männlich'] > 10000) & (df['weiblich'] > 10000) & (df['Todesursachen'] != 'Insgesamt')]
max

,Todesursachen,männlich,weiblich,Insgesamt
5,Neubildungen,127968,109192,237160
6,BösartigeNeubildungen(BN),124006,105062,229068
12,"BN der Leber, der Gallenwege und des Pankreas",16708,14286,30994
15,"BN d. Larynx. d. Trachea, d. Bronchien u.d. Lunge",28332,17608,45940
16,BN der Bronchien und der Lunge,27200,17401,44601
20,BN der Genital- und Harnorgane,25989,16181,42170
31,"Endokrine, Ernährungs- u. Stoffwechselkranheiten",17402,20464,37866
32,Diabetesmellitus,12428,13556,25984
33,PsychischeundVerhaltensstörungen,22848,37142,59990
36,Krankheiten d. Nervensystems u.d. Sinnesorgane,17974,17804,35778


In [227]:
#buat grafik penyebab kematian di jerman sepanjang tahun 2021
import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Bar(name='männlich', x=max['Todesursachen'], y=max['männlich']),
    go.Bar(name='weiblich', x=max['Todesursachen'], y=max['weiblich'])
])
fig.update_layout(
    xaxis_title="Todesursachen",
    yaxis_title="Nilai",
    title="Todesursachen dengan Nilai 'männlich' dan 'weiblich' > 100,000",
    width=800,
    height=900
)

fig.show()

Peringkat penyebab kematian di jerman sepanjang 2021:
1. Krankheiten des Kreislaufsystems
2. 